# Red Calibration for DS Apollo fixture

## Checklist
 - [ ] Calibration Tube
 - [ ] Sekonik C8000
```
1. Setup the sekonik, cam, lamp
1.1 replace batteries in sekonic and preform black calibartion (DONT FORGET to set the ring back to central position)
1.2 attach the sekonik to calibration device, add lamp, run the RedCalib.ino Ardudino project
```
 - [ ] Arduino Fimrware for Red Calibration (arduino link, arduino setup link)
 - [ ] M5 Camera with sekonik mount (cam product link, sekonik holder Blender/step file)
```
2. Connect the camera
2.1 Connetc computer to ssid: M5UV2_305b (passw: 12345678)
2.2 ssh m5stack@10.254.239.1
2.3 sudo wpa_passphrase ssid password (12345678)
2.4 sudo nano /etc/wpa_supplicant.conf (paste in the previous output)
2.5 update camera IP (ie. cam = "http://192.168.0.108/video_feed")
2.6 reboot the camera
2.7 ssh m5stack@10.254.239.1
2.8 get the ip for the camera (ifconfig wlan0 | grep inet)
2.9 set correct webcam address (cam = "http://192.168.2.25/video_feed")
```
 - [ ] Jupyter-lab notebook for the workflow

## The software 
will ask the lamp serial and then prefom RED output measurements from start temp to 85C and output serial_red.csv with temp, lux

In [142]:
cam = "http://10.254.239.1/video_feed"

import cv2
import easyocr
import sys
import os
import pandas as pd
from ipywidgets import widgets, Image, Output
from PIL import Image as Pilimage
import numpy as np

class scope():
    def __init__(self):
        self.reader = easyocr.Reader(['ch_sim','en'])
        
        self.size = [512,512]
        
        self.output = Output()
        self.slider_h = widgets.IntRangeSlider(max=self.size[0], value=(315,450),layout=widgets.Layout(width='90%'),observe=self._update_preview)
        self.slider_v = widgets.IntRangeSlider(max=self.size[1], value=(90,140),layout=widgets.Layout(width='90%'),observe=self._update_preview)
        
        self.offset_x = widgets.IntSlider(max=320//2, min=320//-2, value=0, layout=widgets.Layout(width='90%'),observe=self._update_preview)
        self.offset_y = widgets.IntSlider(max=320//2, min=320//-2, value=0, layout=widgets.Layout(width='90%'),observe=self._update_preview)
        
        self.cropped_display = widgets.Image()
        
        self._update_image()
        
        self.slider_h.observe(self._update_preview)
        self.slider_v.observe(self._update_preview)
        self.offset_x.observe(self._update_preview)
        self.offset_y.observe(self._update_preview)
        
        # Tabbed view for simpler controls
        tab_contents = ['Crop', 'Offset']
        children = [widgets.VBox([
            widgets.Label(value="Width:"),
            self.slider_h,
            widgets.Label(value="Height:"),
            self.slider_v
        ]),widgets.VBox([
            widgets.Label(value="Offset X:"),
            self.offset_x,
            widgets.Label(value="Offset Y:"),
            self.offset_y
        ])]
        
        tab = widgets.Tab()
        tab.children = children
        tab.titles = ['Crop', 'Offset']
        
        self._update_preview()
        
        display(
            tab,
            widgets.Label(value="Out:"),
            self.cropped_display, 
            self.output,
        )
        
    def _update_preview(self, event=0):
        # Crop the display image out of the full capture
        self.cropped = self.image.crop((self.slider_h.value[0],
                                        self.slider_v.value[0],
                                        self.slider_h.value[1],
                                        self.slider_v.value[1]))
        self.cropped_display.value = self.cropped._repr_png_()
        self.cropped_display.format = 'PNG'
    
    def _update_image(self):
        camera = cv2.VideoCapture(cam)
        _, img = camera.read()
        img = Pilimage.fromarray(img)
        img = img.rotate(-90, expand=True)

        self.slider_h.max = img.size[1]
        self.slider_v.max = img.size[0]
        
        self.image = img

        camera.release()
        
    def get_val(self):
        return(self.reader.readtext(np.asarray(self.cropped))[0][1])
        
    

In [143]:
a = scope()

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Label(value='Out:')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x87\x00\x00\x002\x08\x02\x00\x00\x00\xf9\x82_\x9…

Output()

In [144]:
a.get_val()

'3736'

In [3]:
# run measuring loop update every 5 sec, up untill 85C then back to idle
# idle 10% red fan max

In [ ]:
# Once done, show graph and wire file